# 곱셈 계층

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backword(self, dout):
        dx = dout * self.y # x,y 바꿈
        dy = dout * self.x
        
        return dx, dy
    
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy
    
apple = 100
apple_num = 2
tax = 1.1

# 계층 
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price) # 220

# 역전파 
dprice = 1
dapple_price, dtax = mul_tax_layer.backword(dprice)
dapplp, dapple_num = mul_apple_layer.backword(dapple_price)

print(dapplp, dapple_num, dtax) #2.2, 110, 200




220.00000000000003
2.2 110.00000000000001 200


# ReLU

In [2]:
import numpy as np
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        """
        x의 원소값이 0이하인 인덱스는 True, 그 외엔 False
        """
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        """
        순전파 떄 만들어둔 mask를 써서 mask의 원소가 True인 곳에는 상류에서 전파된 dout를 0으로 설정 
        """
        dout[self.mask] = 0 
        dx = dout 
        
        return dx
x = np.array( [[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


# 시그모이드

In [3]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx


# Affine/Softmax 계층 구현하기

배치용 Affine 계층

> 신경망의 순전파 때 수행하는 행렬의 내적을 기하학에서는 어파인 변환이라고 함.

In [4]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        
        self.x = x
        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx

# Softmax-with-Loss 계층

소프트맥스 함수는 입력값을 정규화하며 출력한 것. 신경망에서 정규화하지 않은 출력 결과에서는 소프트맥스 앞의 어파인을 점수라고 함.

> 학습과 추론이 있는데, 추론할 때는 일반적으로 소프트맥스를 사용하지 않음. 신경망을 학습할때는 소프트맥스 계층이 필요함

그림으로 보면 조금 복잡하지만, 미분해보면 쉽다. 젼님이 올린 수식 풀이을 참고.
소프트 맥스 계층은 입력 (a1, a2, a3)를 정규화 하여 (y1,y2,y3)를 출력. CEE 계층은 소프트맥스의 출력 (y1~y3)와 (t1~t3)를 받고 데이터로부터 손실 L을 출력함

소프트 계층의 역전파는 (y1-t1, y2-t2, y3-t3) 결과를 출력함. 이는 소프트맥스 계층의 출력과 정답레이블의 차분. 역전파에선 이 차이인 오차가 앞 계층에 전해지는 것.


정답 레이블이 (0, 1, 0)일 때 소프트맥스 계층이 (0.3, 0.2, 0.5)를 출력했을때. 정답의 확률이 0.2라서 커다란 오차를 전파하게됨

정답 레이블이 (0, 1, 0)일 때 소프트맥스 계층이 (0.01, 0.99, 0)을 출력한 경우엔 작은 오차를 전파하게 됨.

In [5]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size # 전파하는 값을 배치의 수로 나눠서 데이터 1개당 오차를 앞계층으로 전파
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx



# 오차역전파법 구현하기

In [10]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
# from common.gradient import numerical_gradient
from collections import OrderedDict
from dataset.mnist import load_mnist
# from two_layer_net import TwoLayerNet

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

W1:2.56928600433e-13
b1:8.82221936564e-13
W2:8.8173162562e-13
b2:1.20126120162e-10


In [13]:
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:2.24069377139e-13
b1:9.52635327935e-13
W2:8.56118220587e-13
b2:1.19459998837e-10


1. 수치 미분을 써서 구하는 방법 -> 쉽다
2. 해석적으로 수식을 풀어 구하는 방법 -> 어렵다

해석적으로 구하는 방법은 오차역전파법을 이용하여 매개변수가 많아도 효율적으로 계산할 수 있었다. 제대로 구현했는지 검증함. 위의 코드는 예제 코드 5장의 기울기 확인 코드다.

In [11]:

# coding: utf-8
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
# from two_layer_net import TwoLayerNet
# 오차역전파법을 사용한 학습 구현하기

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)


0.113016666667 0.1144
0.9039 0.9071
0.926016666667 0.9268
0.93885 0.9375
0.9468 0.9444
0.953116666667 0.9509
0.958616666667 0.9556
0.963016666667 0.9594
0.966233333333 0.96
0.96835 0.964
0.971466666667 0.9656
0.973566666667 0.9672
0.9746 0.9684
0.977116666667 0.9692
0.977533333333 0.9704
0.979583333333 0.9721
0.980633333333 0.9724


계산 그래프를 이용하면 계산 과정을 시각적으로 파악할 수 있기는 함. 이해하긴 쉬운데, 미분이 낫다..
신경망의 구성 요소를 계층으로 구현하여 기울기를 효율적으로 계산할 수 있다.
오차역전파법 구현이 잘못된 것이 없는지 기울기를 확인하도록 하자.